In [1]:
import pandas as pd
import os
import dataflow
import datetime
import trade_strategy
from dateutil.relativedelta import relativedelta
from trade_logger import TradeLogger, EventType
%matplotlib inline

## 跑一支股票

In [5]:
df = pd.read_excel("./crawl_result/ETF50/1101.xlsx")

In [6]:
df = dataflow.append_all_index(dataflow.preprocess(df))

--
--
--
--
--
--


/Users/rogerlo/Documents/work/code/python/test/finace_crawler/toolbox.py:78: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  rsv_series = df[column].rolling(window).apply(RSV)


In [7]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    money_in_pocket = 100 #(K)
    money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.baseline(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i))
    #print(trade_log)
    #報酬
    diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
    #報酬率
    print("year: {}, diff: {:f}, diff%: {:f}".format((start_date+relativedelta(years=i)).year, diff, diff/money_in_pocket*100))
    

year: 2013, diff: 18.300000, diff%: 18.300000
year: 2014, diff: 0.600000, diff%: 0.600000
year: 2015, diff: -26.420000, diff%: -26.420000
year: 2016, diff: 30.990000, diff%: 30.990000
year: 2017, diff: 5.500000, diff%: 5.500000


In [8]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    money_in_pocket = 100 #(K)
    money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.KD_basic(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i))
    #print(trade_log)
    #報酬
    diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
    #報酬率
    print("year: {}, diff: {:f}, diff%: {:f}".format((start_date+relativedelta(years=i)).year, diff, diff/money_in_pocket*100))
    

year: 2013, diff: 16.100000, diff%: 16.100000
year: 2014, diff: 8.450000, diff%: 8.450000
year: 2015, diff: -16.760000, diff%: -16.760000
year: 2016, diff: 16.100000, diff%: 16.100000
year: 2017, diff: 2.350000, diff%: 2.350000


## 跑ETF 50

In [2]:
dfs = []
for file in os.listdir("./crawl_result/ETF100/"):
    dfs.append(dataflow.append_all_index(dataflow.preprocess(pd.read_excel(f"./crawl_result/ETF100/{file}"))))

/Users/rogerlo/Documents/work/code/python/test/finace_crawler/toolbox.py:78: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  rsv_series = df[column].rolling(window).apply(RSV)


--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--


In [3]:
baseline_row = []

In [4]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    year = (start_date+relativedelta(years=i)).year
    for df in dfs:
        money_in_pocket = 1000 #(K)
        money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.baseline(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i))
        #print(trade_log)
        #報酬
        diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
        #報酬率
        baseline_row.append((df.iloc[0]["stock_name"], year, diff/money_in_pocket*100))
        #print("year: {}, diff: {:f}, diff%: {:f}".format(year, diff, diff/money_in_pocket*100))


In [5]:
KD_row = []

In [6]:
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2013,12,31)
for i in range(5):
    year = (start_date+relativedelta(years=i)).year
    for df in dfs:
        money_in_pocket = 1000 #(K)
        money_in_pocket_result, stock_share, end_price, trade_log = trade_strategy.KD_basic(df, money_in_pocket, start_date+relativedelta(years=i), end_date+relativedelta(years=i), all_in=True, cross=False)
        #print(trade_log)
        #報酬
        diff = (stock_share*end_price+money_in_pocket_result) - money_in_pocket
        #報酬率
        KD_row.append((df.iloc[0]["stock_name"], year, diff/money_in_pocket*100))
        #print("year: {}, diff: {:f}, diff%: {:f}".format(year, diff, diff/money_in_pocket*100))

In [7]:
baseline_df = pd.DataFrame(data=baseline_row, columns=["name", "year", "baseline"])
kd_df = pd.DataFrame(data=KD_row, columns=["name", "year", "kd"])

In [8]:
result_df = pd.merge(baseline_df, kd_df, on=["name","year"])

In [9]:
result_df[result_df["baseline"]>=0]["baseline"].sum(), result_df[result_df["baseline"]>=0]["kd"].sum()

(12008.468, 3322.552)

In [10]:
result_df[result_df["baseline"]<0]["baseline"].sum(), result_df[result_df["baseline"]<0]["kd"].sum()

(-2755.487, -1576.4080000000001)

In [11]:
# baselin法與kd法賺的比例
len(result_df[result_df["baseline"]>=0])/len(result_df), len(result_df[result_df["kd"]>=0])/len(result_df)

(0.652, 0.628)

In [12]:
# 在baseline賺的時候，KD賺的比例
len(result_df[(result_df["baseline"]>0)&(result_df["kd"]>0)])/len(result_df[result_df["baseline"]>0])

0.8235294117647058

In [13]:
# 在baseline賠時，KD賺或持平的比例
len(result_df[(result_df["baseline"]<0)&(result_df["kd"]>=0)])/len(result_df[result_df["baseline"]<0])

0.27011494252873564

In [14]:
# 在baseline賺或持平時，KD賠的比例
len(result_df[(result_df["baseline"]>=0)&(result_df["kd"]<0)])/len(result_df[result_df["baseline"]>=0])

0.18098159509202455